Instalando pacotes

Importação

Dependências para scrapping

In [ ]:

!pip install requests beautifulsoup4 pandas numpy

### Web scraping EMBRAPA
Utilizando parâmetros de URL e na tabela 'tb_base tb_dados'


Dependências para tratamento de dados, caso houvesse maior quantidade de registros, no lugar do levenshtein, utilizar o rapidfuzz

In [ ]:
!pip install fuzzywuzzy python-levenshtein unidecode

Coleta inicial de dados limpos do site da embrapa para inicio da análise,
o script abaixo faz a varredura conjunto a padronização inicial (L = KG, remoção de pontuação de milhas e cabeçalhos, conjunto a adição de contexto de extração)

In [ ]:
# Bibliotecas necessárias para requisição, parsing HTML, manipulação de dados e controle de arquivos
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
import os
from pathlib import Path

# ---------------------- CONFIGURAÇÕES INICIAIS ----------------------

# Define o diretório onde os arquivos CSV serão salvos
DATA_DIR = "data/raw"
# Cria o diretório se ele ainda não existir
Path(DATA_DIR).mkdir(parents=True, exist_ok=True)

# URL base do site da Embrapa Vitibrasil
base_url = "http://vitibrasil.cnpuv.embrapa.br/index.php"
# Intervalo de anos para coleta dos dados
anos = range(2008, 2024)

# Dicionário com as páginas do site a serem raspadas
# Cada entrada contém: nome do arquivo, se o ano é necessário e subopções da página (se houver)
config_paginas = {
    "opt_02": {
        "nome_arquivo": "producao",
        "requer_ano": True,
        "subopcoes": [None]
    },
    "opt_03": {
        "nome_arquivo": "processamento",
        "requer_ano": True,
        "subopcoes": [f"subopt_{i:02d}" for i in range(1, 3)]
    },
    "opt_04": {
        "nome_arquivo": "comercializacao",
        "requer_ano": True,
        "subopcoes": [None]
    },
    "opt_05": {
        "nome_arquivo": "importacao",
        "requer_ano": True,
        "subopcoes": [f"subopt_{i:02d}" for i in range(1, 3)]
    },
    "opt_06": {
        "nome_arquivo": "exportacao",
        "requer_ano": True,
        "subopcoes": [f"subopt_{i:02d}" for i in range(3, 5)]
    }
}

# ---------------------- FUNÇÕES PRINCIPAIS ----------------------

def scrape_tabelas(base_url, params):
    """
    Realiza requisição HTTP ao site com os parâmetros fornecidos
    e retorna as tabelas HTML encontradas com a classe 'tb_base tb_dados'
    """
    try:
        response = requests.get(base_url, params=params, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        tabelas = soup.find_all('table', {'class': 'tb_base tb_dados'})
        return tabelas, soup
    except Exception as e:
        print(f"Erro ao acessar {params}: {str(e)}")
        return [], None

def processar_tabela_com_itens(soup, tabela):
    """
    Processa tabelas que possuem estrutura hierárquica com 'item' e 'subitem'
    Retorna um DataFrame estruturado mantendo a relação entre eles
    """
    linhas = tabela.find_all('tr')
    dados = []
    current_item = None

    for linha in linhas:
        item = linha.find('td', {'class': 'tb_item'})
        if item:
            current_item = item.get_text(strip=True)
            continue

        subitem = linha.find('td', {'class': 'tb_subitem'})
        if subitem and current_item:
            celulas = [td.get_text(strip=True) for td in linha.find_all('td')]
            dados.append([current_item] + celulas)

    if dados:
        num_colunas = max(len(linha) for linha in dados)
        colunas = ['Item'] + [f'Coluna_{i}' for i in range(1, num_colunas)]
        df = pd.DataFrame(dados, columns=colunas)
    else:
        # Caso a tabela não siga o padrão esperado, usa leitura direta
        df = pd.read_html(str(tabela))[0]

    return df

def tratar_valores_nulos(df):
    """
    Substitui valores inválidos ('-', 'nd') por 0 e tenta converter colunas para numérico
    """
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].replace('-', '0')
            df[col] = df[col].replace('nd', '0')
            try:
                df[col] = pd.to_numeric(df[col], errors='ignore')
            except:
                pass
    return df

def arquivo_existe(nome_arquivo):
    """
    Verifica se o arquivo CSV já existe no diretório de saída
    """
    caminho_completo = os.path.join(DATA_DIR, nome_arquivo)
    return os.path.exists(caminho_completo)

# ---------------------- FLUXO PRINCIPAL ----------------------

# Itera sobre as páginas configuradas
for opcao, config in config_paginas.items():
    # Gera o nome do arquivo de saída baseado no nome e no intervalo de anos
    if config["requer_ano"]:
        nome_arquivo = f"{config['nome_arquivo']}_{min(anos)}_{max(anos)}.csv"
    else:
        nome_arquivo = f"{config['nome_arquivo']}.csv"

    # Pula a coleta se o arquivo já foi salvo anteriormente
    if arquivo_existe(nome_arquivo):
        print(f"Arquivo {nome_arquivo} já existe. Pulando coleta...")
        continue

    dfs = []  # Lista para armazenar DataFrames por ano/subopção
    intervalos = anos if config["requer_ano"] else [None]

    # Itera sobre os anos (se aplicável) e subopções da página
    for ano in intervalos:
        for subopcao in config["subopcoes"]:
            params = {"opcao": opcao}
            if ano is not None:
                params["ano"] = ano
            if subopcao:
                params["subopcao"] = subopcao

            print(f"Coletando: opcao={opcao}, ano={ano}, subopcao={subopcao}")
            tabelas, soup = scrape_tabelas(base_url, params)

            for i, tabela in enumerate(tabelas):
                try:
                    # Decide como processar a tabela com base na estrutura do HTML
                    if soup and (soup.find('td', {'class': 'tb_item'}) or tabela.find('td', {'class': 'tb_item'})):
                        df = processar_tabela_com_itens(soup, tabela)
                    else:
                        df = pd.read_html(str(tabela))[0]

                    # Limpeza e tratamento de dados nulos
                    df = tratar_valores_nulos(df)

                    # Adiciona coluna do ano (se aplicável)
                    if ano is not None:
                        df['Ano'] = ano

                    dfs.append(df)
                except Exception as e:
                    print(f"Erro ao processar tabela: {str(e)}")

            sleep(2)  # Pausa entre requisições para respeitar o servidor

    # Salva o arquivo CSV final se houve coleta
    if dfs:
        caminho_completo = os.path.join(DATA_DIR, nome_arquivo)
        df_final = pd.concat(dfs, ignore_index=True)
        df_final.to_csv(caminho_completo, index=False)
        print(f"Dados salvos em {caminho_completo}")

print("Coleta concluída!")


Dataframe para padronização e conversão da coluna em lista para a aplicação do fuzzywuzzy

In [ ]:
# CSV contendo nome de países baseados no IBGE
paises_padronizados = pd.read_csv('https://balanca.economia.gov.br/balanca/bd/tabelas/PAIS.csv', encoding='latin1', sep=';').rename(columns={'NO_PAIS': 'Pais'})

# Extrair lista de nomes padronizados
paises = paises_padronizados['Pais'].tolist()

In [ ]:
# CSV contendo nome de países baseados no IBGE
paises_padronizados_2 = pd.read_csv('http://raw.githubusercontent.com/AndiDittrich/GeoIP-Country-Lists/refs/heads/master/GeoLite2/GeoLite2-Country-CSV_20150407/GeoLite2-Country-Locations-pt-BR.csv', encoding='UTF-8', sep=',').rename(columns={'country_name': 'Pais'})
# paises_padronizados_2
# Extrair lista de nomes padronizados
paises_2 = paises_padronizados_2['Pais'].tolist()

Implementações do fuzzywuzzy

Aplicando FuzzyWuzzy para preparar a base para futuras inclusões de bases de dados de outros países

In [ ]:
import pandas as pd
import unidecode
from fuzzywuzzy import process, fuzz

def normalizar(texto):
    """Remove acentos, coloca em minúsculo e tira espaços extras."""
    if pd.isnull(texto):
        return ""
    return unidecode.unidecode(texto.lower().strip())

def fuzzy_match_dataframe(
    df_origem,
    referencia,
    coluna_origem='Pais',
    coluna_referencia='Pais',
    threshold=85,
    scorer=fuzz.token_sort_ratio,
    correcoes_manuais=None,
    excluir_valores=None  # lista de valores a ignorar
):
    """
    Faz fuzzy matching entre países com suporte a:
    - Correções manuais via dicionário
    - Lista de valores a excluir do matching
    """

    # Normaliza lista de exclusão
    excluir_norm = set()
    if excluir_valores:
        excluir_norm = set(normalizar(v) for v in excluir_valores)

    # Verifica referência
    if isinstance(referencia, pd.DataFrame):
        lista_ref = referencia[coluna_referencia].dropna().tolist()
    else:
        lista_ref = referencia

    lista_ref_norm = [normalizar(p) for p in lista_ref]
    ref_dict = dict(zip(lista_ref_norm, lista_ref))  # Para recuperar forma original

    correcoes_norm = {}
    if correcoes_manuais:
        correcoes_norm = {
            normalizar(k): v for k, v in correcoes_manuais.items()
        }

    def corrigir(pais):
        pais_norm = normalizar(pais)

        # Ignora se estiver na lista de exclusão
        if pais_norm in excluir_norm:
            return pd.Series([None, None])

        # Substituição direta se houver
        if pais_norm in correcoes_norm:
            return pd.Series([correcoes_norm[pais_norm], 100])

        # Aplica fuzzy
        match_norm, score = process.extractOne(pais_norm, lista_ref_norm, scorer=scorer)
        match_final = ref_dict.get(match_norm) if score >= threshold else None
        return pd.Series([match_final, score])

    # Aplica correções
    df_origem[['País_corrigido', 'threshold']] = df_origem[coluna_origem].apply(corrigir)

    # Retorna apenas registros com correspondência válida
    return df_origem[df_origem['País_corrigido'].notnull()].copy()


Nomes onde algorítmo sofreu algum problema para funcionar, devido as
características dos registros originais, esses sendo filtrados e trabalhados
manualmente

In [ ]:
correcoes = {
    "Alemanha, República Democrática": "Alemanha", "Eslovaca, Republica": 'Eslováquia', 'Países Baixos': 'Holanda'
}

excluir = [
    "Total", "Outros(1)"
]

resultado = fuzzy_match_dataframe(
    exportacao,
    paises_2,
    threshold=50,
    correcoes_manuais=correcoes,
    excluir_valores=excluir
)


In [ ]:
resultado


Checagem de consistência, para incrementar o tratamento, se necessário
(Como podemos ver Coveite apresenta uma threshold grande e acaba por não atender adequadamente, sendo necessária a inclusão no bloco acima)

In [ ]:
resultado_filtrado = resultado[["Pais", "País_corrigido", "threshold"]].rename(columns={"País_corrigido": "Pais_fuzzy"})
resultado_filtrado_distinto = resultado_filtrado.drop_duplicates(subset="Pais").sort_values(by="threshold", ascending=True).reset_index(drop=True)
resultado_filtrado_distinto

In [ ]:
resultado.to_csv('exportacao_2008_2023_corrigido.csv', index=False)